In [129]:
import pandas as pd
import asyncio
import numpy as np
import aiohttp
import requests
import re

boys = pd.read_excel('mnpnombres.xlsx', sheet_name=0, header=4, usecols="A:AI", nrows=378, skiprows=[5])
boys.rename( columns={'Unnamed: 0':'Nombre'}, inplace=True )
# remove composed names
boys = boys[boys.Nombre.str.count(' ') == 0]
boys['origen'] = None
boys

,Nombre,2020,2019,2018,2017,2016,2015,2014,2013,2012,...,1995,1994,1993,1992,1991,1990,1989,1988,1987,origen
0,AARON,121,225,152,172,197,206,200,179,191,...,53,38,38,47,35,37,36,47,31,None
1,ABEL,46,44,54,41,55,57,52,58,52,...,26,38,31,40,64,39,56,45,40,None
2,ABRAHAM,34,39,51,39,58,47,57,44,43,...,28,23,26,33,32,25,37,35,39,None
3,ACHRAF,6,14,5,6,6,4,4,11,9,...,6,3,2,0,1,0,0,0,0,None
4,ADAM,103,125,99,110,108,120,113,126,136,...,4,4,3,1,2,5,0,3,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,ZAKARIA,6,8,15,11,15,8,9,16,15,...,2,1,1,0,0,1,0,0,0,None
374,ZAYD,8,3,5,3,1,1,6,3,3,...,0,0,0,0,0,0,0,0,0,None
375,ZEUS,5,2,5,2,2,1,2,2,2,...,0,0,2,3,6,7,7,7,6,None
376,ZIAD,8,5,6,8,6,3,3,1,1,...,0,0,0,0,0,1,0,0,0,None


In [130]:
girls = pd.read_excel('mnpnombres.xlsx',  sheet_name=3,header=4, usecols="A:AI", nrows=454, skiprows=[5])
girls.rename( columns={'Unnamed: 0':'Nombre'}, inplace=True )
girls['origen'] = None
# remove composed names
girls = girls[girls.Nombre.str.count(' ') == 0]
girls

,Nombre,2020,2019,2018,2017,2016,2015,2014,2013,2012,...,1995,1994,1993,1992,1991,1990,1989,1988,1987,origen
0,ABBY,9,3,2,4,1,2,1,0,0,...,0,0,0,0,0,0,0,0,0,None
1,ABIGAIL,19,17,24,20,17,13,19,21,13,...,1,2,3,5,9,7,5,7,6,None
2,ABRAR,6,2,6,3,3,4,1,1,1,...,0,0,0,0,0,0,0,0,0,None
3,ABRIL,161,73,136,124,82,92,66,52,61,...,1,0,1,0,1,0,0,0,1,None
4,ADA,21,14,18,11,24,28,25,19,14,...,7,3,1,2,3,2,1,1,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,YASMINE,5,5,4,5,3,7,9,6,9,...,2,0,1,0,1,3,0,1,0,None
449,ZAHARA,6,2,8,2,0,2,2,2,2,...,0,0,3,1,0,4,3,2,1,None
450,ZAINAB,7,5,8,6,12,6,4,3,10,...,1,1,0,0,0,0,0,0,0,None
451,ZAIRA,18,44,15,19,19,25,24,24,35,...,12,8,17,17,16,17,22,19,13,None


In [148]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

# Obtain origin async
async def get(url, session, reg_exp, old_value):
    try:
        if old_value is not None:
            return old_value
        async with session.get(url=url) as response:
            resp = await response.text()
            
            txt = cleanhtml(str(resp))
            txt = txt.replace(r"\xc3\x81", "a")
            txt = txt.replace(r"\xc3\xad", "i")
            txt = txt.replace(r"\r\n", " ")
                    
            origen = re.search(reg_exp, txt)
            if origen is None:
                return None
            ori = origen.group(1)
            print("Successfully got url {} with origin {}.".format(url, ori))            
            return ori
        
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))


In [132]:

# Sync method to obtain the origin. Used for testing
def get_origen(name):
    url = get_url_w(name)    
    print(url)
    r = requests.get(url)    
    reg_exp =get_exp_w(name)
    txt = cleanhtml(str(r.content))
    txt = txt.replace(r"\xc3\x81", "a")
    txt = txt.replace(r"\xc3\xad", "i")    
    txt = txt.replace(r"\r\n", " ")
    origen = re.search(reg_exp, txt)
    if origen is None:
        return txt, None
    return txt, origen.group(1)    



In [134]:
# Several origin pages providers to parse
ori = re.compile('(?<=de origen )(\w+)')
def get_exp_w(name):
    return ori
def get_url_w(name):
    return f'https://www.google.com/search?q=origen+{name}.html'

ori = re.compile('(?<=de origen )(\w+)')
def get_exp(name):
    return ori
def get_url(name):
    return f'https://www.misabueso.com/nombres/nombre_{name}.html'

def get_exp2(name):
    return re.compile(f'(?<=Origen del nombre {name.capitalize()}: )(\w+)')
def get_url2(name):
    return f"https://www.bekiapadres.com/nombres-bebe/{name}"


def get_exp3(name):
    return ori
def get_url3(name):
    return f"https://elsignificadodelnombre.com/{name}/"

def get_exp4(name):
    return ori
def get_url4_boys(name):
    return f"https://www.nombresparamibebe.com/nombres-de-nino/{name}/"

def get_url4_girls(name):
    return f"https://www.nombresparamibebe.com/nombres-de-nina/{name}/"


def get_exp5(name):
    return re.compile(f'(?<=Origen del nombre {name.capitalize()}: )(\w+)')
def get_url5(name):
    return f"https://www.guiadelnino.com/nombres-de-bebes/{name}/"

def get_exp6(name):
    return re.compile(f'(?<=El origen de este nombre es )(\w+)')
def get_url6(name):
    return f"https://es.namespedia.com/details/{name}/"

def get_exp7(name):
    return re.compile(f'(?<=Origen del nombre {name.capitalize()}:)(\w+)')
def get_url7(name):
    return f"https://www.conmishijos.com/nombres/{name}/"

def get_exp8(name):
    return ori
def get_url8(name):
    return f"https://www.bekiapadres.com/nombres-bebe/{name}"

def get_exp9(name):
    return ori
def get_url9(name):
    return f"https://www.mundoprimaria.com/nombres-ninos-ninas-significado/significado-{name}"

def get_exp10(name):
    return re.compile(f'(?<=Origen de {name.capitalize()}:         )(\w+)')
def get_url10(name):
    return f"https://mibebeyyo.elmundo.es/nombres-bebes/{name}"





In [135]:
# Call the async for all the names without origin filled in the dataset for girls and for boys
async def call_with_func(g_url, g_exp):
    async with aiohttp.ClientSession() as session:
        girls['origen'] = await asyncio.gather(*[get(g_url(row['Nombre']), session, g_exp(row['Nombre']), row['origen']) for index, row in girls.iterrows()])

    async with aiohttp.ClientSession() as session:
        boys['origen'] = await asyncio.gather(*[get(g_url(row['Nombre']), session, g_exp(row['Nombre']), row['origen']) for index, row in boys.iterrows()])    


In [136]:
# Call the origin providers to fill the value
await call_with_func(get_url, get_exp)
await call_with_func(get_url2, get_exp2)
await call_with_func(get_url7, get_exp7)
await call_with_func(get_url9, get_exp9)
await call_with_func(get_url4_boys, get_exp4)
await call_with_func(get_url4_girls, get_exp4)
await call_with_func(get_url3, get_exp3)

# Clean some values
boys['origen'] = boys['origen'].str.replace('Significado','')
girls['origen'] = girls['origen'].str.replace('Significado','')
boys['origen'] = boys['origen'].str.upper()
girls['origen'] = girls['origen'].str.upper()

Successfully got url https://www.misabueso.com/nombres/nombre_AFRICA.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_ALEXANDRA.html with origin Griego.
Successfully got url https://www.misabueso.com/nombres/nombre_AINARA.html with origin vasco.
Successfully got url https://www.misabueso.com/nombres/nombre_ANA.html with origin hebreo.
Successfully got url https://www.misabueso.com/nombres/nombre_ARIA.html with origin Griego.
Successfully got url https://www.misabueso.com/nombres/nombre_AISHA.html with origin Árabe.
Successfully got url https://www.misabueso.com/nombres/nombre_ARANTXA.html with origin vasco.
Successfully got url https://www.misabueso.com/nombres/nombre_ALEX.html with origin Griego.
Successfully got url https://www.misabueso.com/nombres/nombre_AMAYA.html with origin vasco.
Successfully got url https://www.misabueso.com/nombres/nombre_AMINA.html with origin árabe.
Successfully got url https://www.misabueso.com/nombres/nombre_ANGELA.htm

Successfully got url https://www.misabueso.com/nombres/nombre_JUDITH.html with origin Hebreo.
Successfully got url https://www.misabueso.com/nombres/nombre_AMARA.html with origin Mauritania.
Successfully got url https://www.misabueso.com/nombres/nombre_ALBA.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_ADA.html with origin Hebreo.
Successfully got url https://www.misabueso.com/nombres/nombre_LAILA.html with origin Egipcio.
Successfully got url https://www.misabueso.com/nombres/nombre_KIRA.html with origin Griego.
Successfully got url https://www.misabueso.com/nombres/nombre_LAURA.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_ANGELICA.html with origin griego.
Successfully got url https://www.misabueso.com/nombres/nombre_LEA.html with origin Hebreo.
Successfully got url https://www.misabueso.com/nombres/nombre_LAYLA.html with origin hebreo.
Successfully got url https://www.misabueso.com/nombres/nombre_LEAH.ht

Successfully got url https://www.misabueso.com/nombres/nombre_JULIA.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_JULIETA.html with origin Griego.
Successfully got url https://www.misabueso.com/nombres/nombre_ISABEL.html with origin Hebreo.
Successfully got url https://www.misabueso.com/nombres/nombre_ZOE.html with origin Griego.
Successfully got url https://www.misabueso.com/nombres/nombre_AMIRA.html with origin Árabe.
Successfully got url https://www.misabueso.com/nombres/nombre_ANTONELLA.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_ALEJANDRA.html with origin Griego.
Successfully got url https://www.misabueso.com/nombres/nombre_KEYLA.html with origin Gaélico.
Successfully got url https://www.misabueso.com/nombres/nombre_ALICIA.html with origin germánico.
Successfully got url https://www.misabueso.com/nombres/nombre_KARLA.html with origin Germánico.
Successfully got url https://www.misabueso.com/nombres/n

Successfully got url https://www.misabueso.com/nombres/nombre_MARCO.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_MARCELO.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_MARCOS.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_MATIAS.html with origin Hebreo.
Successfully got url https://www.misabueso.com/nombres/nombre_MARTIN.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_MATEO.html with origin Hebreo.
Successfully got url https://www.misabueso.com/nombres/nombre_MAURO.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_MIGUEL.html with origin hebreo.
Successfully got url https://www.misabueso.com/nombres/nombre_MAX.html with origin Latín.
Successfully got url https://www.misabueso.com/nombres/nombre_MOISES.html with origin Egipcio.
Successfully got url https://www.misabueso.com/nombres/nombre_NOE.html

Successfully got url https://www.bekiapadres.com/nombres-bebe/NURIA with origin Español.
Successfully got url https://www.bekiapadres.com/nombres-bebe/TRIANA with origin Árabe.
Successfully got url https://www.bekiapadres.com/nombres-bebe/ROMA with origin Italiano.
Successfully got url https://www.bekiapadres.com/nombres-bebe/SIENA with origin Anglosajón.
Successfully got url https://www.bekiapadres.com/nombres-bebe/BRAIS with origin Latino.
Successfully got url https://www.bekiapadres.com/nombres-bebe/ADONAY with origin Hebreo.
Successfully got url https://www.bekiapadres.com/nombres-bebe/BORJA with origin Árabe.
Successfully got url https://www.bekiapadres.com/nombres-bebe/BOSCO with origin Latino.
Successfully got url https://www.bekiapadres.com/nombres-bebe/IBAI with origin Vasco.
Successfully got url https://www.bekiapadres.com/nombres-bebe/ARITZ with origin Vasco.
Successfully got url https://www.bekiapadres.com/nombres-bebe/ADAY with origin Hebreo.
Successfully got url https://w

Successfully got url https://www.mundoprimaria.com/nombres-ninos-ninas-significado/significado-YERAI with origin guache.
Successfully got url https://www.nombresparamibebe.com/nombres-de-nino/ALESSIA/ with origin griego.
Successfully got url https://www.nombresparamibebe.com/nombres-de-nino/AYLA/ with origin turco.
Successfully got url https://www.nombresparamibebe.com/nombres-de-nino/BJORN/ with origin escandinavo.
Successfully got url https://www.nombresparamibebe.com/nombres-de-nino/OLIVER/ with origin francés.
Successfully got url https://www.nombresparamibebe.com/nombres-de-nina/INDARA/ with origin hindú.
Successfully got url https://www.nombresparamibebe.com/nombres-de-nina/JARA/ with origin eslavo.
Successfully got url https://www.nombresparamibebe.com/nombres-de-nina/PAUL/ with origin inglés.
Successfully got url https://elsignificadodelnombre.com/ANE/ with origin hebreo.
Successfully got url https://elsignificadodelnombre.com/AGATHA/ with origin griego.
Successfully got url ht

Successfully got url https://elsignificadodelnombre.com/YASSIN/ with origin árabe.
Successfully got url https://elsignificadodelnombre.com/ARON/ with origin hebreo.
Successfully got url https://elsignificadodelnombre.com/YOUSSEF/ with origin hebreo.
Successfully got url https://elsignificadodelnombre.com/YOUSEF/ with origin hebreo.
Successfully got url https://elsignificadodelnombre.com/YANIS/ with origin hebreo.
Successfully got url https://elsignificadodelnombre.com/ZIAD/ with origin africano.
Successfully got url https://elsignificadodelnombre.com/ZAKARIA/ with origin hebreo.
Successfully got url https://elsignificadodelnombre.com/MASSIMO/ with origin latín.
Successfully got url https://elsignificadodelnombre.com/SULAIMAN/ with origin africano.
Successfully got url https://elsignificadodelnombre.com/SALMAN/ with origin árabe.
Successfully got url https://elsignificadodelnombre.com/LIAN/ with origin latín.
Successfully got url https://elsignificadodelnombre.com/ZAYD/ with origin afri

In [144]:
# Change order of years columns
years = [a for a in range(1987,2021)]
new_index = ['Nombre', 'origen'] + years
boys = boys.reindex(new_index, axis = 1)
girls = girls.reindex(new_index, axis = 1)
dfs =[boys, girls]

In [84]:
# Helper methods to manually asign origins to names
def change_origen(dfs, old, new):
    for df in dfs:
        df['origen'].mask(df['origen'] == old, new, inplace=True)

def assign_origen(df, name, value):
    df['origen'] = np.where(df['Nombre'] == name, value,  df['origen'])

In [158]:
# Manually assign origin that are wrongly assigned
assign_origen(boys, "IÑIGO", "VASCO")
assign_origen(boys, "YERAY", "GUACHE")
assign_origen(boys, "HARON", "ÁRABE")
assign_origen(boys, "TIAGO", "BÍBLICO")
assign_origen(boys, "GUZMAN", "GERMANO")
assign_origen(boys, "HUGO", "GERMANO")
assign_origen(boys, "LUCCA", "ITALIANO")
assign_origen(boys, "THEO", "GRIEGO")
assign_origen(boys, "NEO", "GRIEGO")
assign_origen(boys, "YASER", "ÁRABE")
assign_origen(boys, "YASSER", "HEBREO")
assign_origen(boys, "YASSIR", "ÁRABE")
assign_origen(boys, "ACHRAF", "ÁRABE")
assign_origen(boys, "YOUNES", "ÁRABE")
assign_origen(boys, "KYLIAN", "GAÉLICO")
assign_origen(boys, "ARAY", "HEBREO")
assign_origen(boys, "MAIKEL", "HEBREO")
assign_origen(boys, "MAXIMILIANO", "LATINO")
assign_origen(boys, "ALEJANDRO", "GRIEGO")
assign_origen(boys, "NICOLÁS", "GRIEGO")
assign_origen(boys, "JORGE", "GRIEGO")
assign_origen(boys, "GONZALO", "GERMANO")
assign_origen(boys, "RODRIGO", "GERMANO")
assign_origen(boys, "ALBERTO", "GERMANO")
assign_origen(boys, "LUIS", "GERMANO")
assign_origen(boys, "RAUL", "GERMANO")
assign_origen(boys, "SANTIAGO", "HEBREO")
assign_origen(boys, "LUCAS", "LATINO")
assign_origen(boys, "CARLOS", "LATINO")
assign_origen(boys, "ANGEL", "LATINO")
assign_origen(boys, "HECTOR", "GRIEGO")
assign_origen(boys, "ALFONSO", "GRIEGO")
assign_origen(boys, "JAVIER", "VASCO")
assign_origen(boys, "DARIO", "LATINO")
assign_origen(boys, "KAI", "HAWAIANO")
assign_origen(girls, "ARIANA", "GRIEGO")
assign_origen(girls, "ARIANNA", "GRIEGO")
assign_origen(girls, "SOPHIA", "GRIEGO")
assign_origen(girls, "MARWA", "ÁRABE")
assign_origen(girls, "KAWTAR", "ÁRABE")
assign_origen(girls, "NAYARA", "ÁRABE")
assign_origen(girls, "NAYLA", "ÁRABE")
assign_origen(girls, "NAHIARA", "ÁRABE")
assign_origen(girls, "ISRAA", "ÁRABE")
assign_origen(girls, "LUA", "LATINO")
assign_origen(girls, "AINHARA", "VASCO")
assign_origen(girls, "ARIADNE", "GRIEGO")
assign_origen(girls, "AVRIL", "LATINO")
assign_origen(girls, "KHLOE", "GRIEGO")
assign_origen(girls, "MEGAN", "GALÉS")
assign_origen(girls, "ALESSANDRA", "ITALIANO")
assign_origen(girls, "VALLE", "LATINO")
assign_origen(girls, "YAREL", "HEBREO")

In [159]:
# Unify origin names
change_origen(dfs, 'LATÍN', 'LATINO')
change_origen(dfs, 'GUANCHE', 'GUACHE')
change_origen(dfs, 'CANARIO', 'GUACHE')
change_origen(dfs, 'GERMÁNICO', 'GERMANO')


In [160]:
# Export data
boys.to_csv('boys.csv')
girls.to_csv('girls.csv')